In [180]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import random
import json 
from pandas.io.json import json_normalize

df=pd.read_csv("/Users/veillonpaul-armand/Documents/Projets/Data/Carto/CP_conso.csv")



In [47]:
with open("/Users/veillonpaul-armand/Documents/Projets/Data/LA_FISCALITE_ET_LES_DEPENSES_PUBLIQUES_short.json") as json_file:
    json_data = json.load(json_file) # or geojson.load(json_file)
df_GN=json_normalize(json_data)

#Cleaning CP, le CP est renseigné par l'utilisateur, ne garder que les valeurs plausibles
df_GN["authorZipCode"]=pd.to_numeric(df_GN["authorZipCode"])
df_GN.loc[df_GN["authorZipCode"]<1000,"authorZipCode"]=np.nan
df_GN.loc[df_GN["authorZipCode"]>98999,"authorZipCode"]=np.nan
df_GN["DEP"]=df_GN["authorZipCode"]/1000
df_GN["DEP"]=df_GN["DEP"].fillna(0).astype('int')

#Cleaning formated answers
answers=["Reculer l'âge de la retraite","Augmenter le temps de travail","Augmenter les impôts",
        "Revoir les conditions d'attribution de certaines aides sociales", "ISF"]
label=["a","b","c","d"]

for i in range(4):
    df_GN["Fisca_1"+label[i]]=df_GN["Fisca_1"].str.contains(answers[i])
    all_ref=df_GN.groupby(["authorZipCode"])["Fisca_1"+label[i]].mean()
    all_ref=all_ref.to_frame()
    df=df.merge(all_ref,how="left", right_on="authorZipCode",left_on="Code Postal")

df=df.loc[~df["Fisca_1a"].isnull()]    

quant=sorted(list(set(df["quantile_rev"])))
plot_data = pd.DataFrame({'quantile': quant,})

for j in label:
    plot_data["Fisca_1"+j]=0
    for i in quant:
        plot_data.loc[plot_data['quantile']==i,"Fisca_1"+j]=sum(df.loc[df["quantile_rev"]==i,"Fisca_1"+j]*
                                    df.loc[df["quantile_rev"]==i,"POP_CP"])/sum(df.loc[df["quantile_rev"]==i,"POP_CP"])
TUU=sorted(list(set(df['CP_TUU'])))
plot_data_TUU = pd.DataFrame({'TUU': TUU,})

for j in label:
    plot_data_TUU["Fisca_1"+j]=0
    for i in TUU:
        plot_data_TUU.loc[plot_data['TUU']==i,question[i]["name"]+question[i]["label"][j]]=sum(df.loc[df["CP_TUU"]==i,"Fisca_1"+j]*
                                    df.loc[df["CP_TUU"]==i,"POP_CP"])/sum(df.loc[df["CP_TUU"]==i,"POP_CP"])
       

Fisca_1a
False    68015
True     17916
Name: Fisca_1a, dtype: int64
Fisca_1b
False    64466
True     21465
Name: Fisca_1b, dtype: int64
Fisca_1c
False    82601
True      3330
Name: Fisca_1c, dtype: int64
Fisca_1d
False    41306
True     44625
Name: Fisca_1d, dtype: int64


In [181]:
with open("/Users/veillonpaul-armand/Documents/Projets/Data/LA_TRANSITION_ECOLOGIQUE_short.json") as json_file:
    json_data = json.load(json_file) # or geojson.load(json_file)
df_GN=json_normalize(json_data)

#Cleaning CP, le CP est renseigné par l'utilisateur, ne garder que les valeurs plausibles
df_GN["authorZipCode"]=pd.to_numeric(df_GN["authorZipCode"])
df_GN.loc[df_GN["authorZipCode"]<1000,"authorZipCode"]=np.nan
df_GN.loc[df_GN["authorZipCode"]>98999,"authorZipCode"]=np.nan
df_GN["DEP"]=df_GN["authorZipCode"]/1000
df_GN["DEP"]=df_GN["DEP"].fillna(0).astype('int')


question=[{'name':"Ecolo_0","answers":["La pollution de l'air", "Les dérèglements climatiques", "L'érosion du littoral",
         "La biodiversité et la disparition de certaines espèces"],'label':["a","b","c","d"]},
        {"name":"Ecolo_2","answers":["Oui","Non"],"label":["a","b"]},
        {"name":"Ecolo_4","answers":["Oui","Non"],"label":["a","b"]},
        {"name":"Ecolo_8","answers":["Oui","Non"],"label":["a","b"]},
        {"name":"Ecolo_10","answers":["Oui","Non","Je n'utilise pas la voiture"],"label":["a","b","c"]},
        {"name":"Ecolo_12","answers":["Les transports en commun","Le covoiturage","L'auto partage","Le transport à la demande","Le vélo"],"label":["a","b","c","d","e"]},
        ]

quant=sorted(list(set(df["quantile_rev"])))
plot_data = pd.DataFrame({'quantile': quant,})
plot_data_plus = pd.DataFrame({'quantile': quant,})
TUU=sorted(list(set(df['CP_TUU'])))
plot_data_TUU = pd.DataFrame({'TUU': TUU,})

for i in range(6):
    l=len(question[i]["answers"])
    for j in range(l):
        df_GN[question[i]["name"]+question[i]["label"][j]]=df_GN[question[i]["name"]].str.contains(question[i]["answers"][j])
        df_GN["none"]=df_GN[question[i]["name"]].str.contains('|'.join(question[i]["answers"]))
        temp=df_GN[df_GN["none"]]
        all_ref=temp.groupby(["authorZipCode"])[question[i]["name"]+question[i]["label"][j]].mean()
        all_ref=all_ref.to_frame()
        df=df.merge(all_ref,how="left", right_on="authorZipCode",left_on="Code Postal")
        
        df=df.loc[~df[question[i]["name"]+question[i]["label"][j]].isnull()]   
        
        plot_data[question[i]["name"]+question[i]["label"][j]]=0
        plot_data_plus[question[i]["name"]+question[i]["label"][j]]=0
        for k in quant:
            plot_data.loc[plot_data['quantile']==k,question[i]["name"]+question[i]["label"][j]]=sum(df.loc[df["quantile_rev"]==k,question[i]["name"]+question[i]["label"][j]]*
                                    df.loc[df["quantile_rev"]==k,"POP_CP"])/sum(df.loc[df["quantile_rev"]==k,"POP_CP"])
            plot_data_plus.loc[plot_data['quantile']==k,question[i]["name"]+question[i]["label"][j]]=sum(df.loc[df["quantile_CSP+"]==k,question[i]["name"]+question[i]["label"][j]]*
                                    df.loc[df["quantile_CSP+"]==k,"POP_CP"])/sum(df.loc[df["quantile_CSP+"]==k,"POP_CP"])
        
        plot_data_TUU[question[i]["name"]+question[i]["label"][j]]=0
        for k in TUU:
            plot_data_TUU.loc[plot_data_TUU['TUU']==k,question[i]["name"]+question[i]["label"][j]]=sum(df.loc[df["CP_TUU"]==k,question[i]["name"]+question[i]["label"][j]]*
                                    df.loc[df["CP_TUU"]==k,"POP_CP"])/sum(df.loc[df["CP_TUU"]==k,"POP_CP"])
   

In [183]:
plot_data_TUU

,TUU,Ecolo_0a,Ecolo_0b,Ecolo_0c,Ecolo_0d,Ecolo_2a,Ecolo_2b,Ecolo_4a,Ecolo_4b,Ecolo_8a,Ecolo_8b,Ecolo_10a,Ecolo_10b,Ecolo_10c,Ecolo_12a,Ecolo_12b,Ecolo_12c,Ecolo_12d,Ecolo_12e
0,0.0,0.242214,0.409146,0.014205,0.337328,0.651650,0.348350,0.937786,0.062214,0.577401,0.422599,0.201765,0.707621,0.090614,0.709590,0.330089,0.205418,0.338586,0.276956
1,1.0,0.262695,0.437831,0.007652,0.297296,0.658442,0.341558,0.942472,0.057528,0.587476,0.412524,0.250329,0.650878,0.098794,0.718360,0.293002,0.167870,0.311383,0.348542
2,2.0,0.263893,0.417512,0.014354,0.306777,0.649056,0.350944,0.945009,0.054991,0.587634,0.412366,0.290998,0.591452,0.117550,0.727091,0.283504,0.168104,0.283142,0.349027
3,3.0,0.264116,0.436965,0.013426,0.287051,0.648032,0.351968,0.934219,0.065781,0.583142,0.416858,0.316718,0.557488,0.125795,0.737410,0.269510,0.175904,0.290111,0.370662
4,4.0,0.261982,0.441228,0.015679,0.282908,0.643429,0.356571,0.922745,0.077255,0.619631,0.380369,0.333928,0.505192,0.160880,0.732355,0.261259,0.139467,0.279650,0.404375
5,5.0,0.289837,0.430010,0.012206,0.269110,0.640998,0.359002,0.937499,0.062501,0.601843,0.398157,0.406876,0.429079,0.164045,0.722100,0.272644,0.154508,0.268146,0.454008
6,6.0,0.315098,0.400393,0.008346,0.277339,0.680668,0.319332,0.948761,0.051239,0.594765,0.405235,0.498594,0.357355,0.144051,0.739753,0.267632,0.176839,0.298795,0.467035
7,7.0,0.309198,0.414845,0.006621,0.272957,0.663878,0.336122,0.943034,0.056966,0.610159,0.389841,0.511822,0.233520,0.254658,0.776252,0.253753,0.179900,0.263095,0.515674
8,8.0,0.326303,0.405032,0.007528,0.262909,0.641141,0.358859,0.932658,0.067342,0.600016,0.399984,0.481038,0.141017,0.377945,0.777720,0.220874,0.183220,0.293369,0.495158


In [132]:
plot_data_TUU

,TUU,Ecolo_0a,Ecolo_0b,Ecolo_0c,Ecolo_0d,Ecolo_2a,Ecolo_2b,Ecolo_4a,Ecolo_4b,Ecolo_8a,Ecolo_8b,Ecolo_10a,Ecolo_10b,Ecolo_10c,Ecolo_12a,Ecolo_12b,Ecolo_12c,Ecolo_12d,Ecolo_12e
0,0.0,0.154952,0.262085,0.008611,0.208100,0.605973,0.319223,0.872236,0.057729,0.520719,0.378853,0.182925,0.656046,0.082182,0.430045,0.203421,0.130020,0.207261,0.172277
1,1.0,0.167887,0.272479,0.005115,0.187254,0.622961,0.317518,0.887485,0.052507,0.531855,0.373897,0.232028,0.610150,0.089185,0.443152,0.182614,0.108448,0.194878,0.210941
2,2.0,0.169432,0.263979,0.008728,0.192696,0.608356,0.324568,0.881847,0.050557,0.533740,0.365111,0.264465,0.547495,0.107718,0.450635,0.174923,0.106702,0.170870,0.220975
3,3.0,0.167324,0.273522,0.008215,0.180145,0.593976,0.317819,0.850304,0.060239,0.510998,0.356584,0.279375,0.499039,0.115423,0.439972,0.165414,0.105096,0.171692,0.219539
4,4.0,0.176558,0.293430,0.009932,0.185440,0.603887,0.332706,0.866169,0.071634,0.543343,0.333044,0.304455,0.469822,0.147082,0.439452,0.160225,0.083225,0.165275,0.241103
5,5.0,0.182095,0.275613,0.007961,0.174608,0.594779,0.330081,0.866638,0.057308,0.534702,0.354978,0.370115,0.393812,0.149190,0.422942,0.157460,0.089019,0.160230,0.263409
6,6.0,0.207281,0.254621,0.004920,0.179789,0.636219,0.295564,0.887038,0.047094,0.516326,0.353047,0.457632,0.331165,0.130495,0.422254,0.148970,0.100117,0.164829,0.265385
7,7.0,0.195240,0.261170,0.004411,0.170661,0.608050,0.308062,0.862063,0.051825,0.528990,0.338864,0.460842,0.213643,0.228476,0.398846,0.131849,0.092263,0.137678,0.260979
8,8.0,0.204437,0.254699,0.004730,0.163153,0.587742,0.324902,0.848214,0.060796,0.512906,0.343852,0.427192,0.124982,0.341193,0.372968,0.107977,0.088833,0.141001,0.239376


In [146]:
plot_data_plus

,quantile,Ecolo_0a,Ecolo_0b,Ecolo_0c,Ecolo_0d,Ecolo_2a,Ecolo_2b,Ecolo_4a,Ecolo_4b,Ecolo_8a,Ecolo_8b,Ecolo_10a,Ecolo_10b,Ecolo_10c,Ecolo_12a,Ecolo_12b,Ecolo_12c,Ecolo_12d,Ecolo_12e
0,0.0,0.196557,0.274766,0.007076,0.167952,0.582888,0.325148,0.858052,0.048475,0.507570,0.349993,0.427197,0.164622,0.299271,0.382876,0.116059,0.090163,0.141237,0.252783
1,0.1,0.185304,0.257357,0.003704,0.174748,0.602941,0.310963,0.860802,0.051590,0.526058,0.338740,0.450597,0.198200,0.251222,0.401351,0.133896,0.094310,0.135086,0.260655
2,0.2,0.194199,0.268858,0.007856,0.168993,0.618550,0.295920,0.862946,0.052484,0.517444,0.351585,0.420899,0.287932,0.196277,0.412501,0.145059,0.099659,0.145635,0.260386
3,0.3,0.172912,0.258834,0.007907,0.189033,0.608824,0.317350,0.864247,0.064756,0.525540,0.347652,0.380733,0.367959,0.169104,0.430392,0.145563,0.095933,0.165416,0.258379
4,0.4,0.185381,0.268693,0.005272,0.183363,0.610121,0.314128,0.861130,0.061659,0.521741,0.357593,0.361391,0.418471,0.133149,0.424696,0.150339,0.097226,0.161831,0.230394
5,0.5,0.188680,0.280290,0.005864,0.167758,0.604582,0.327193,0.875656,0.059422,0.541026,0.347393,0.325647,0.446890,0.144103,0.422989,0.166764,0.095356,0.176821,0.222956
6,0.6,0.197389,0.253296,0.005163,0.183604,0.613006,0.321959,0.879951,0.051786,0.535266,0.360678,0.337891,0.446873,0.134937,0.416964,0.163868,0.104407,0.189812,0.220743
7,0.7,0.181968,0.266849,0.006498,0.183286,0.611401,0.321882,0.861478,0.062741,0.530217,0.352703,0.287721,0.494683,0.126770,0.422811,0.178796,0.100736,0.170412,0.213622
8,0.8,0.171928,0.263981,0.006383,0.201342,0.607832,0.327318,0.875174,0.059531,0.539665,0.359801,0.250044,0.563074,0.114044,0.433512,0.171848,0.103461,0.180744,0.205331
9,0.9,0.171976,0.249503,0.006268,0.181342,0.624325,0.312599,0.885781,0.050282,0.550613,0.353881,0.214540,0.622446,0.092530,0.433970,0.175231,0.111579,0.169859,0.204295


In [154]:
with open("/Users/veillonpaul-armand/Documents/Projets/Data/DEMOCRATIE_ET_CITOYENNETE_short.json") as json_file:
    json_data = json.load(json_file) # or geojson.load(json_file)
df_GN=json_normalize(json_data)

#Cleaning CP, le CP est renseigné par l'utilisateur, ne garder que les valeurs plausibles
df_GN["authorZipCode"]=pd.to_numeric(df_GN["authorZipCode"])
df_GN.loc[df_GN["authorZipCode"]<1000,"authorZipCode"]=np.nan
df_GN.loc[df_GN["authorZipCode"]>98999,"authorZipCode"]=np.nan
df_GN["DEP"]=df_GN["authorZipCode"]/1000
df_GN["DEP"]=df_GN["DEP"].fillna(0).astype('int')


question=[{"name":"Cit_1","answers":["Oui","Non"],"label":["a","b"]},
        {"name":"Cit_4","answers":["Une bonne chose","Une mauvaise chose","Je ne sais pas"],"label":["a","b","c"]},
        {"name":"Cit_7","answers":["Oui","Non"],"label":["a","b"]},
        {"name":"Cit_10","answers":["Oui","Non"],"label":["a","b"]},
        {"name":"Cit_13","answers":["Oui","Non","Je ne sais pas"],"label":["a","b","c"]},
        {"name":"Cit_17","answers":["Oui","Non","Je ne sais pas"],"label":["a","b","c"]}
         ]

quant=sorted(list(set(df["quantile_rev"])))
plot_data = pd.DataFrame({'quantile': quant,})
plot_data_plus = pd.DataFrame({'quantile': quant,})
TUU=sorted(list(set(df['CP_TUU'])))
plot_data_TUU = pd.DataFrame({'TUU': TUU,})

for i in range(6):
    l=len(question[i]["answers"])
    for j in range(l):
        df_GN[question[i]["name"]+question[i]["label"][j]]=df_GN[question[i]["name"]].str.contains(question[i]["answers"][j])
        df_GN["none"]=df_GN[question[i]["name"]].str.contains('|'.join(question[i]["answers"]))
        temp=df_GN[df_GN["none"]]
        all_ref=temp.groupby(["authorZipCode"])[question[i]["name"]+question[i]["label"][j]].mean()
        all_ref=all_ref.to_frame()
        df=df.merge(all_ref,how="left", right_on="authorZipCode",left_on="Code Postal")
        
        df=df.loc[~df[question[i]["name"]+question[i]["label"][j]].isnull()]   
        
        plot_data[question[i]["name"]+question[i]["label"][j]]=0
        plot_data_plus[question[i]["name"]+question[i]["label"][j]]=0
        for k in quant:
            plot_data.loc[plot_data['quantile']==k,question[i]["name"]+question[i]["label"][j]]=sum(df.loc[df["quantile_rev"]==k,question[i]["name"]+question[i]["label"][j]]*
                                    df.loc[df["quantile_rev"]==k,"POP_CP"])/sum(df.loc[df["quantile_rev"]==k,"POP_CP"])
            plot_data_plus.loc[plot_data['quantile']==k,question[i]["name"]+question[i]["label"][j]]=sum(df.loc[df["quantile_CSP+"]==k,question[i]["name"]+question[i]["label"][j]]*
                                    df.loc[df["quantile_CSP+"]==k,"POP_CP"])/sum(df.loc[df["quantile_CSP+"]==k,"POP_CP"])
        
        plot_data_TUU[question[i]["name"]+question[i]["label"][j]]=0
        for k in TUU:
            plot_data_TUU.loc[plot_data_TUU['TUU']==k,question[i]["name"]+question[i]["label"][j]]=sum(df.loc[df["CP_TUU"]==k,question[i]["name"]+question[i]["label"][j]]*
                                    df.loc[df["CP_TUU"]==k,"POP_CP"])/sum(df.loc[df["CP_TUU"]==k,"POP_CP"])
   

In [155]:
plot_data

,quantile,Cit_1a,Cit_1b,Cit_4a,Cit_4b,Cit_4c,Cit_7a,Cit_7b,Cit_10a,Cit_10b,Cit_13a,Cit_13b,Cit_13c,Cit_17a,Cit_17b,Cit_17c
0,0.0,0.546183,0.343244,0.706806,0.118849,0.084877,0.641593,0.239616,0.750069,0.156922,0.491207,0.212223,0.176144,0.628885,0.083387,0.0
1,0.1,0.498309,0.336514,0.684039,0.132401,0.084885,0.645647,0.221513,0.735079,0.148085,0.469288,0.217715,0.178422,0.577209,0.093087,0.0
2,0.2,0.524837,0.333658,0.690550,0.118541,0.073113,0.594394,0.257492,0.732791,0.141051,0.461470,0.209849,0.184016,0.581464,0.095652,0.0
3,0.3,0.501001,0.347167,0.655156,0.126981,0.083686,0.627890,0.213900,0.706073,0.154254,0.452633,0.216245,0.176265,0.577109,0.086473,0.0
4,0.4,0.530648,0.357404,0.705064,0.111682,0.091841,0.637091,0.236544,0.746651,0.152678,0.451667,0.238163,0.190504,0.580945,0.096895,0.0
5,0.5,0.506241,0.354201,0.683711,0.117180,0.089841,0.617628,0.232584,0.744167,0.136624,0.457931,0.228178,0.180681,0.593101,0.084541,0.0
6,0.6,0.535033,0.330487,0.682991,0.113887,0.094022,0.619677,0.242193,0.737360,0.150493,0.467071,0.211114,0.191872,0.594455,0.091199,0.0
7,0.7,0.508587,0.362026,0.688381,0.116708,0.090432,0.637004,0.220516,0.743886,0.146924,0.473885,0.207012,0.188249,0.587235,0.088288,0.0
8,0.8,0.506612,0.357118,0.661507,0.120818,0.103519,0.624885,0.223298,0.727321,0.151441,0.449432,0.234894,0.176455,0.590626,0.084246,0.0
9,0.9,0.467979,0.365940,0.624330,0.141874,0.095724,0.603090,0.215861,0.670787,0.178928,0.428043,0.249202,0.165800,0.571295,0.092968,0.0


In [175]:
with open("/Users/veillonpaul-armand/Documents/Projets/Data/ORGANISATION_DE_LETAT_ET_DES_SERVICES_PUBLICS_short.json") as json_file:
    json_data = json.load(json_file) # or geojson.load(json_file)
df_GN=json_normalize(json_data)

#Cleaning CP, le CP est renseigné par l'utilisateur, ne garder que les valeurs plausibles
df_GN["authorZipCode"]=pd.to_numeric(df_GN["authorZipCode"])
df_GN.loc[df_GN["authorZipCode"]<1000,"authorZipCode"]=np.nan
df_GN.loc[df_GN["authorZipCode"]>98999,"authorZipCode"]=np.nan
df_GN["DEP"]=df_GN["authorZipCode"]/1000
df_GN["DEP"]=df_GN["DEP"].fillna(0).astype('int')


question=[{"name":"Etat_1","answers":["Oui","Non"],"label":["a","b"]},
          {"name":"Etat_3","answers":["Oui","Non"],"label":["a","b"]},
          {"name":"Etat_6","answers":["Oui","Non"],"label":["a","b"]},
          {"name":"Etat_7","answers":["Oui","Non"],"label":["a","b"]},
          {"name":"Etat_11","answers":["Oui","Non"],"label":["a","b"]},
          {"name":"Etat_12","answers":["Oui","Non"],"label":["a","b"]},
          {"name":"Etat_15","answers":["Oui","Non"],"label":["a","b"]},
          {"name":"Etat_17","answers":["Oui","Non"],"label":["a","b"]},
         ]

quant=sorted(list(set(df["quantile_rev"])))
plot_data = pd.DataFrame({'quantile': quant,})
plot_data_plus = pd.DataFrame({'quantile': quant,})
TUU=sorted(list(set(df['CP_TUU'])))
plot_data_TUU = pd.DataFrame({'TUU': TUU,})

for i in range(8):
    l=len(question[i]["answers"])
    for j in range(l):
        df_GN[question[i]["name"]+question[i]["label"][j]]=df_GN[question[i]["name"]].str.contains(question[i]["answers"][j])
        df_GN["none"]=df_GN[question[i]["name"]].str.contains('|'.join(question[i]["answers"]))
        temp=df_GN[df_GN["none"]]
        all_ref=temp.groupby(["authorZipCode"])[question[i]["name"]+question[i]["label"][j]].mean()
        all_ref=all_ref.to_frame()
        all_ref=all_ref.replace([np.inf,-np.inf],np.nan).dropna()
        df=df.merge(all_ref,how="left", right_on="authorZipCode",left_on="Code Postal")
        
        df=df.loc[~df[question[i]["name"]+question[i]["label"][j]].isnull()]   
        
        plot_data[question[i]["name"]+question[i]["label"][j]]=0
        plot_data_plus[question[i]["name"]+question[i]["label"][j]]=0
        
        for k in quant:
            plot_data.loc[plot_data['quantile']==k,question[i]["name"]+question[i]["label"][j]]=sum(df.loc[df["quantile_rev"]==k,question[i]["name"]+question[i]["label"][j]]*
                                    df.loc[df["quantile_rev"]==k,"POP_CP"])/sum(df.loc[df["quantile_rev"]==k,"POP_CP"])
            plot_data_plus.loc[plot_data['quantile']==k,question[i]["name"]+question[i]["label"][j]]=sum(df.loc[df["quantile_CSP+"]==k,question[i]["name"]+question[i]["label"][j]]*
                                    df.loc[df["quantile_CSP+"]==k,"POP_CP"])/sum(df.loc[df["quantile_CSP+"]==k,"POP_CP"])
        
        plot_data_TUU[question[i]["name"]+question[i]["label"][j]]=0
        for k in TUU:
            plot_data_TUU.loc[plot_data_TUU['TUU']==k,question[i]["name"]+question[i]["label"][j]]=sum(df.loc[df["CP_TUU"]==k,question[i]["name"]+question[i]["label"][j]]*
                                    df.loc[df["CP_TUU"]==k,"POP_CP"])/sum(df.loc[df["CP_TUU"]==k,"POP_CP"])
       

In [172]:
df_GN[question[i]["name"]].str.contains('|'.join(question[i]["answers"]))

0        False
1        False
2        False
3        False
4         True
5         True
6         True
7         True
8         True
9         True
10       False
11       False
12       False
13        True
14       False
15        True
16       False
17        True
18        True
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27        True
28       False
29        True
         ...  
51588     True
51589     True
51590     True
51591     True
51592     True
51593     True
51594    False
51595     True
51596     True
51597     True
51598     True
51599     True
51600     True
51601     True
51602    False
51603    False
51604    False
51605     True
51606     True
51607     True
51608     True
51609     True
51610     True
51611     True
51612     True
51613     True
51614     True
51615    False
51616     True
51617     True
Name: Etat_17, Length: 51618, dtype: bool

In [163]:
question[i]["answers"]

['Oui', 'Non']

In [179]:
plot_data_TUU

,TUU,Etat_1a,Etat_1b,Etat_3a,Etat_3b,Etat_6a,Etat_6b,Etat_7a,Etat_7b,Etat_11a,Etat_11b,Etat_12a,Etat_12b,Etat_15a,Etat_15b,Etat_17a,Etat_17b
0,0.0,0.517479,0.482521,0.508598,0.491402,0.255278,0.744722,0.613210,0.386790,0.767823,0.232177,0.095734,0.904266,0.727040,0.272960,0.891126,0.108874
1,1.0,0.509084,0.490916,0.526577,0.473423,0.262770,0.737230,0.611937,0.388063,0.785415,0.214585,0.086431,0.913569,0.710073,0.289927,0.905379,0.094621
2,2.0,0.524568,0.475432,0.595058,0.404942,0.274941,0.725059,0.642846,0.357154,0.786904,0.213096,0.091999,0.908001,0.700265,0.299735,0.879984,0.120016
3,3.0,0.521345,0.478655,0.603280,0.396720,0.275351,0.724649,0.625498,0.374502,0.781944,0.218056,0.095011,0.904989,0.713368,0.286632,0.902933,0.097067
4,4.0,0.505302,0.494698,0.621328,0.378672,0.262555,0.737445,0.637048,0.362952,0.804593,0.195407,0.079981,0.920019,0.706859,0.293141,0.896709,0.103291
5,5.0,0.515098,0.484902,0.670439,0.329561,0.306220,0.693780,0.664038,0.335962,0.794276,0.205724,0.081586,0.918414,0.700846,0.299154,0.886025,0.113975
6,6.0,0.489122,0.510878,0.709735,0.290265,0.268129,0.731871,0.658849,0.341151,0.783601,0.216399,0.072885,0.927115,0.696408,0.303592,0.903745,0.096255
7,7.0,0.505375,0.494625,0.775153,0.224847,0.277778,0.722222,0.715281,0.284719,0.793339,0.206661,0.083240,0.916760,0.711013,0.288987,0.896550,0.103450
8,8.0,0.483454,0.516546,0.739336,0.260664,0.305595,0.694405,0.715686,0.284314,0.799572,0.200428,0.076334,0.923666,0.705014,0.294986,0.907568,0.092432
